In [12]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

In [13]:
import traitlets
from ipywidgets import Controller
import socket
import threading
import json

class ControllerClient(traitlets.HasTraits):
    change = traitlets.Dict()
    
    @traitlets.validate('change')
    def _clip_change(self, proposal):
        return proposal['value']

    
    def __init__(self, server_ip, server_port):
        self.controller = controller
        self.server_ip = server_ip
        self.server_port = server_port
        self.connect()
        self.setup_trait_links()
    
    def connect(self):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((self.server_ip, self.server_port))
        
    def setup_trait_links(self):        
        def transform(t):
            def f(x):
                return {
                    "value": x,
                    "type": t,
                }
            def g(x):
                return {
                    "value": -x,
                    "type": t,
                }
            if t == "steering":
                return f
            return g
        
        traitlets.dlink((self.controller.axes[2], 'value'), (self, 'change'), transform=transform("steering"))
        traitlets.dlink((self.controller.axes[1], 'value'), (self, 'change'), transform=transform("throttle"))
    
    @traitlets.observe('change')
    def _on_change(self, d):
        # Send the changed axis value to the server
        print(f"Sending message to server: {d}")
        msg = {
            "name": d["name"],
            "new": d["new"],
        }
        try:
            self.sock.sendall(json.dumps(msg).encode('ascii'))
        except Exception as e:
            print("send failed with error. restarting the socket")
#             print(e)
            self.reset_connection()
        
    def reset_connection(self):
        self.close()
        self.connect()
    
    def close(self):
        self.sock.close()

client = ControllerClient(server_ip='localhost', server_port=9999)



Sending message to server: {'name': 'change', 'old': traitlets.Undefined, 'new': {'value': 0.0, 'type': 'steering'}, 'owner': <__main__.ControllerClient object at 0x7b0f6c7a9d50>, 'type': 'change'}
Sending message to server: {'name': 'change', 'old': {'value': 0.0, 'type': 'steering'}, 'new': {'value': -0.0, 'type': 'throttle'}, 'owner': <__main__.ControllerClient object at 0x7b0f6c7a9d50>, 'type': 'change'}
